# Note

This repository is built with the intention of reproducible science - you can viewing a copy of the data used, all the results - plots, tables and notebooks. However, you can also run everything from scratch, and generate the results yourself.

So, if you would like to delete all results generated by the authors, and re-run everything to download the data, extract all features and run the models, and generate all plots, please do the following:

Delete the directories:
- ```data/```
- ```results/```

# 1. Download Chen et. al's catalog from Zenodo

In [3]:
import os
import requests
import wget
import tqdm
import zipfile
import sys
import numpy as np
import pandas as pd
import subprocess
from tqdm.auto import tqdm

In [6]:
if not os.path.exists("data/Table2.txt"):
    print("Downloading Chen et. al's table from Zenodo")

    url = "https://zenodo.org/records/3886372/files/Table2.txt.zip"
    response = requests.get(url, stream=True)

    total_size_in_bytes = int(response.headers.get("content-length", 0))
    block_size = 1024  # 1 Kibibyte
    progress_bar = tqdm.tqdm(total=total_size_in_bytes, unit="iB", unit_scale=True)

    with open("data/Table2.txt.zip", "wb") as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong")

    zip_path = "data/Table2.txt.zip"
    extract_path = "data/"

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)
    print("Unzipped the file successfully.")

    os.remove(zip_path)
    print("Deleted the zip file.")

else:
    print("File already exists: data/Table2.txt")

File already exists: data/Table2.txt


# 2. Choose objects

### INCOMPLETE

Refer to ```siddharth/05 - Classification Experiments/01 create_parentdataset.ipynb``` and make new code based on that.

# 3. Download light curves for objects in Chen et al's catalog

### INCOMPLETE

Refer to ```phoenix/ztf_download/hidden_data_multiclass/``` and convert the 5 notebooks/py to a single notebook below. 

---

Finally, we are left with:
- ```data/X_df.csv```
- ```data/HIDDENX_df_binary.csv```
- ```data/HIDDENX_df_multiclass.csv```
- ```data/HIDDENX_df_ovr.csv```
- ```data/y_df.csv```
- ```data/HIDDENy_df_binary.csv```
- ```data/HIDDENy_df_multiclass.csv```
- ```data/HIDDENy_df_ovr.csv```
